<a href="https://colab.research.google.com/github/achanhon/coursdeeplearningcolab/blob/master/geometrie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch


def extractCropOld(ABCD, img, s=256, inner=2048):
    EPS, img = 0.1, img
    _, H, W = img.shape
    rA, cA, rB, cB, rC, cC, rD, cD = ABCD

    tmp = (torch.arange(inner).float() + 0.5) / inner
    u = tmp.view(-1, 1).expand(inner, inner)
    v = tmp.view(1, -1).expand(inner, inner)
    mask = (u + v <= 1.001).float()
    maskbis = (u + v <= 1.001).float()

    rP = (rB - rA) * u + (rD - rA) * v + rA
    rPc = torch.clamp(rP, EPS, H - EPS)
    cP = (cB - cA) * u + (cD - cA) * v + cA
    cPc = torch.clamp(cP, EPS, W - EPS)
    rP, cP = rPc.long().flatten(), cPc.long().flatten()
    out1 = img[:, rP, cP].view(3, inner, inner) * mask.view(1, inner, inner)

    rP = (rD - rC) * u + (rB - rC) * v + rC
    rPc = torch.clamp(rP, EPS, H - EPS)
    cP = (cD - cC) * u + (cB - cC) * v + cC
    cPc = torch.clamp(cP, EPS, W - EPS)
    rP, cP = rPc.long().flatten(), cPc.long().flatten()
    out2 = img[:, rP, cP].view(3, inner, inner) * mask.view(1, inner, inner)
    out2 = torch.flip(out2, dims=[1, 2])
    maskbis = torch.flip(maskbis, dims=[0, 1])

    mask = (mask + maskbis + EPS).view(1, 1, inner, inner)
    out = (out1 + out2).view(1, 3, inner, inner) / mask
    out = torch.nn.functional.interpolate(out, size=(s, s), mode="bilinear")
    return out[0]

def extractCropNew(ABCD, img, s=256, inner=2048):
    EPS, img = 0.1, img
    _, H, W = img.shape
    rA, cA, rB, cB, rC, cC, rD, cD = ABCD

    tmp = (torch.arange(inner).float() + 0.5) / inner
    u = tmp.view(-1, 1).expand(inner, inner)
    v = tmp.view(1, -1).expand(inner, inner)
    mask = (u + v <= 1.001).float()
    maskbis = (u + v <= 1.001).float()

    rP = (rB - rA) * u + (rD - rA) * v + rA
    rPc = torch.clamp(rP, EPS, H - EPS)
    cP = (cB - cA) * u + (cD - cA) * v + cA
    cPc = torch.clamp(cP, EPS, W - EPS)
    rP, cP = rPc.long().flatten(), cPc.long().flatten()
    out1 = img[:, rP, cP].view(3, inner, inner) * mask.view(1, inner, inner)

    rP = (rD - rC) * u + (rB - rC) * v + rC
    rPc = torch.clamp(rP, EPS, H - EPS)
    cP = (cD - cC) * u + (cB - cC) * v + cC
    cPc = torch.clamp(cP, EPS, W - EPS)
    rP, cP = rPc.long().flatten(), cPc.long().flatten()
    out2 = img[:, rP, cP].view(3, inner, inner) * mask.view(1, inner, inner)
    out2 = torch.flip(out2, dims=[1, 2])
    maskbis = torch.flip(maskbis, dims=[0, 1])

    mask = (mask + maskbis + EPS).view(1, 1, inner, inner)
    out = (out1 + out2).view(1, 3, inner, inner) / mask
    out = torch.nn.functional.interpolate(out, size=(s, s), mode="bilinear")
    return out[0]